In [1]:
'''
This notebook cleans csv files downloaded from Citi Bike. 
'''

'\nThis notebook cleans csv files downloaded from Citi Bike. \n'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# plotting-- for future use
plt.style.use('default')
sns.set_palette("husl")

# loading
data_dir = Path("../data/raw")
csv_files = list(data_dir.glob("*.csv"))
print(f"Available data files: {[f.name for f in csv_files]}")

# load first file, explicit index
df = pd.read_csv(csv_files[0], index_col='ride_id', low_memory=False)
print(f"Dataset shape: {df.shape}")
df.head()

Available data files: ['202402-citibike-tripdata.csv', '202401-citibike-tripdata.csv', '202403-citibike-tripdata.csv']
Dataset shape: (2121501, 12)


,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
ride_id,,,,,,,,,,,,
B2B980E0EAE1D6F1,classic_bike,2024-02-25 20:25:40.894,2024-02-25 20:43:58.504,Greenwich St & Hubert St,5470.10,Hudson Blvd W & W 36 St,6611.07,40.721319,-74.010065,40.756765,-73.999714,member
1069DDA1FED20568,classic_bike,2024-02-21 22:21:38.446,2024-02-21 22:40:12.259,Greenwich St & Hubert St,5470.10,Hudson Blvd W & W 36 St,6611.07,40.721319,-74.010065,40.756765,-73.999714,member
B58850AF6F2D8BD5,electric_bike,2024-02-14 08:31:14.609,2024-02-14 08:42:30.427,Mercer St & Bleecker St,5679.05,W 20 St & 10 Ave,6306.01,40.727068,-73.996554,40.745686,-74.005141,member
D46E6C5A69048E11,electric_bike,2024-02-05 08:42:25.999,2024-02-05 08:56:26.899,E 20 St & FDR Dr,5886.13,E 74 St & 1 Ave,6953.08,40.733155,-73.975561,40.768974,-73.954823,member
707AF4CF2C7834C2,electric_bike,2024-02-08 11:13:15.969,2024-02-08 11:18:44.259,E 20 St & FDR Dr,5886.13,Pitt St & Stanton St,5406.04,40.733184,-73.975525,40.719261,-73.981780,member


In [3]:
# overview data

# column names for reference
print("Column names:")
print(df.columns.tolist())

# review. correct data types next if necessary.
print("\nData types:")
print(df.dtypes)

# determine if these are necessary to keep for analytics-- volume, granularity
print("\nMissing values:")
print(df.isnull().sum())

Column names:
['rideable_type', 'started_at', 'ended_at', 'start_station_name', 'start_station_id', 'end_station_name', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual']

Data types:
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

Missing values:
rideable_type            0
started_at               0
ended_at                 0
start_station_name    1866
start_station_id      1866
end_station_name      4768
end_station_id        4788
start_lat                0
start_lng                0
end_lat                607
end_lng                607
member_casual            0
dtype: int64


In [4]:
# change data types for dates and objects
# use coerce to avoid raising failures 
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce') 
df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce') 

# change objects to string
df[['rideable_type', 'start_station_name', 'end_station_name', 'member_casual']] = df[['rideable_type', 'start_station_name', 'end_station_name', 'member_casual']].astype('string')

In [5]:
# count missing values & percentages & flag with text
missing_data = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
})
missing_data = missing_data[missing_data['Missing_Count'] > 0]  # only show columns with missing counts
print(missing_data)

max_missing_pct = missing_data['Missing_Percentage'].max() # percentages not counts
print(f"Maximum missing data percentage: {max_missing_pct:.3f}%")

if max_missing_pct < 1.0:
    print("✓ Excellent data quality: All missing data is below 1% threshold.")
    print("  Missing data should not affect analysis results.")

                    Missing_Count  Missing_Percentage
start_station_name           1866            0.087957
start_station_id             1866            0.087957
end_station_name             4768            0.224747
end_station_id               4788            0.225689
end_lat                       607            0.028612
end_lng                       607            0.028612
Maximum missing data percentage: 0.226%
✓ Excellent data quality: All missing data is below 1% threshold.
  Missing data should not affect analysis results.


In [6]:
# drop rows with nulls & confirm counts
df_clean = df.dropna().copy()
df_clean.reset_index(drop=False, inplace=True)
df_clean.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [7]:
# export cleaned data for future use
# using parquet to maintain data types 
df_clean.to_parquet('../data/processed/cleaned_citibike_data.parquet')
